# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,41.05,84,75,43.73,TF,1702525225
1,1,bethel,41.3712,-73.4140,34.93,58,0,5.75,US,1702525170
2,2,longyearbyen,78.2186,15.6401,31.84,74,75,4.61,SJ,1702525241
3,3,puerto natales,-51.7236,-72.4875,38.62,78,70,4.81,CL,1702525225
4,4,san antonio de pale,-1.4014,5.6325,79.86,82,18,8.72,GQ,1702525220


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
import hvplot.pandas
import pandas as pd

# Sample data (replace this with your data)
data = pd.DataFrame({
    'latitude': [37.7749],
    'longitude': [-122.4194],
    'city': ['San Francisco']
})

# Configure the map plot using hvplot
map_plot = data.hvplot.points(
    geo=True, 
    x='longitude', 
    y='latitude', 
    hover_cols=['city'], 
    title='Sample Map Plot'
)

# Display the map
map_plot

c:\Users\fhali\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Points   [longitude,latitude]   (city)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Assume 'city_data_df' is your DataFrame containing city data

# Narrow down cities that fit criteria (example: Max Temp <= 25 and Wind Speed <= 10)
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] <= 25) & (city_data_df['Wind Speed'] <= 10)]

# Drop any rows with null values
cleaned_df = filtered_cities_df.dropna()

# Display sample data
cleaned_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,udachny,66.4167,112.4000,-13.56,83,100,8.86,RU,1702525232
22,22,anadyr,64.7500,177.4833,10.90,79,0,8.95,RU,1702525238
42,42,klyuchi,52.2667,79.1667,-4.67,70,98,8.23,RU,1702525293
49,49,bilibino,68.0546,166.4372,-12.84,99,86,5.70,RU,1702525239
51,51,tiksi,71.6872,128.8694,-2.83,100,100,4.61,RU,1702525235


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Assume 'northern_hemi_df' is your DataFrame containing Northern Hemisphere data

# Use the Pandas copy function to create a new DataFrame
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
6,udachny,RU,66.4167,112.4000,83,
22,anadyr,RU,64.7500,177.4833,79,
42,klyuchi,RU,52.2667,79.1667,70,
49,bilibino,RU,68.0546,166.4372,99,
51,tiksi,RU,71.6872,128.8694,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 5000  # Replace with your desired radius
params = {
    'apiKey': '024cbb47b30d45c797302653f08654ab',
    'categories': 'accommodation',
    'limit': 1  # Limit to 1 result per request
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in cleaned_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']  # Replace 'Lat' with the actual column name in your DataFrame for latitude
    lng = row['Lng']  # Replace 'Lng' with the actual column name in your DataFrame for longitude
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        cleaned_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        cleaned_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{cleaned_df.loc[index, 'City']} - nearest hotel: {cleaned_df.loc[index, 'Hotel Name']}")

# Display sample data
cleaned_df

Starting hotel search
udachny - nearest hotel: Вилюй
anadyr - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
tiksi - nearest hotel: Арктика
ust'-charyshskaya pristan' - nearest hotel: Гостиница
vorgashor - nearest hotel: No hotel found
qaqortoq - nearest hotel: 60* North Greenland
tokmok - nearest hotel: маг.нурмат
iskateley - nearest hotel: Авантаж
chegdomyn - nearest hotel: Бурея
susuman - nearest hotel: No hotel found
kirensk - nearest hotel: Ивушка
val-d'or - nearest hotel: Quality Inn & Suites
college - nearest hotel: Billie's Backpackers Hostel
pont rouge - nearest hotel: No hotel found
kalevala - nearest hotel: Отель "ВелТ"
kaa-khem - nearest hotel: No hotel found
nagqu - nearest hotel: No hotel found
hami - nearest hotel: No hotel found
gadzhiyevo - nearest hotel: Оазис
iqaluit - nearest hotel: Frobisher Inn
polyarnyy - nearest hotel: No hotel found
berezovyy - nearest hotel: No hotel found
pembroke - nearest hotel

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
6,6,udachny,66.4167,112.4000,-13.56,83,100,8.86,RU,1702525232,Вилюй
22,22,anadyr,64.7500,177.4833,10.90,79,0,8.95,RU,1702525238,No hotel found
42,42,klyuchi,52.2667,79.1667,-4.67,70,98,8.23,RU,1702525293,No hotel found
49,49,bilibino,68.0546,166.4372,-12.84,99,86,5.70,RU,1702525239,No hotel found
51,51,tiksi,71.6872,128.8694,-2.83,100,100,4.61,RU,1702525235,Арктика
...,...,...,...,...,...,...,...,...,...,...,...
504,504,khandyga,62.6667,135.6000,-40.11,100,92,1.72,RU,1702525338,Апельсин
515,515,nelidovo,56.2211,32.7774,21.74,96,100,7.16,RU,1702525484,Центральная
525,525,chengde,40.9725,117.9361,19.36,76,99,3.96,CN,1702525485,紫御国际假日酒店
528,528,severo-yeniseyskiy,60.3725,93.0408,-15.95,95,100,6.06,RU,1702525251,Актолик


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Check the column names in the DataFrame
print(cleaned_df.columns)

# Configure the map plot
map_plot = cleaned_df.hvplot.points(
    x='Lng',  # Replace with the actual longitude column name
    y='Lat',   # Replace with the actual latitude column name
    title='Map Plot',
    size=10,  # Adjust the size as needed
)

# Display the map
map_plot

Index(['Unnamed: 0', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Country', 'Date', 'Hotel Name'],
      dtype='object')


:Points   [Lng,Lat]